In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew


load_dotenv(find_dotenv())

True

In [17]:
from crewai_tools import BaseTool, CSVSearchTool

csvs = [r'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_1.csv', 
        r'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_2.csv', 
        r'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_3.csv', 
        r'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_4.csv', 
        r'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_5.csv', 
        r'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_5.csv']


csv_tools = []
for csv in csvs:
    csv_tools.append(CSVSearchTool(csv=csv))

class TableProcessingTool(BaseTool):
    name: str = "Table processing tool"
    description: str = "This tool "

    def _run(self, argument: str) -> str:
        # Implementation goes here
        return "Result from custom tool"

In [18]:
from crewai import Agent

class TableProcessingAgent():
    def CSV_agent(self):
        return Agent(
            role='Table data extractor',
            goal='Query and extract data from a structured table from in CSV format',
            tools=csv_tools,
            backstory='''As a Table data extractor, your mission is to extract relevant information from a CSV table. 
            The data in the table was extracted using an OCR and the headers might not be accurate.
            You must do your best to derive the correct information from tables that might not be perfect''',
            verbose=True,
        )
    def table_text_agent(self):
            return Agent(
            role='Table text data extractor',
            goal='Extract data from an unstructured table in CSV format',
            backstory='''As a Table text data extractor, your mission is to derive relevant information from a table formatted as CSV. 
            The information you gather will be compared to that of the specialized structured CSV table data extractor agent.
            In addition to that, you will also be tasked with finding out information which was not included in the structured tables or the specialized structured CSV table data extractor agent could not find''',
            verbose=True,
        )

In [19]:
from crewai import Task

class ProcessPayslipTablesTask():
    def get_salary_from_tables_task(self, agent):
        return Task(
            description='''Given the extracted tables from the employee's payslip analyze them and return the salary of the employee''',
            expected_output="A numerical value of the salary",
            agent=agent,
        )

In [20]:
agents = TableProcessingAgent()
csv_agent = agents.CSV_agent()
text_agent = agents.table_text_agent()

tasks = ProcessPayslipTablesTask()
get_salary_csv = tasks.get_salary_from_tables_task(csv_agent)
get_salary_text = tasks.get_salary_from_tables_task(text_agent)

In [21]:
from crewai import Crew

# Create Crew responsible for Copy
crew = Crew(
	agents=[
		csv_agent,
		text_agent,
	],
	tasks=[
		get_salary_csv,
		get_salary_text,
	]
)

2024-03-23 21:33:59,650 - 34212 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [22]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
To find the salary of the employee, I need to search for relevant keywords such as "salary", "wage", "pay", "income", etc. in the CSV files. I will start with the first CSV file.

Action: 
Search a CSV's content

Action Input: 
{"search_query": "salary"} 
Doc content has not changed. Skipping creating chunks and embeddings
 

Relevant Content:
: , STATEMENT: , OF
BULLETIN DE: , EARNINGS
PAIE:

: , STATEMENT: , OF
BULLETIN DE: , EARNINGS
PAIE: 2.591,88

: , STATEMENT: HOURS
HEURES, OF
BULLETIN DE: RATE
TAUX, EARNINGS
PAIE: AMOUNT
MONTANT

Thought: 
The observation from the first CSV file does not provide a clear salary information. The content seems to be in French and somewhat jumbled. It mentions an "amount" of 2.591,88, but it's not clear if this is the salary. Let's search the remaining CSV files to gather more data.

Action: 
Search a CSV's content

Action Input: 
{"search_query": "salary"} 
 

I tried reusing the same input, I must stop 